In [2]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

In [3]:
folder = "assets/hpc_out/5_1stm_oj_0_0_os_0_85/"
folder = "assets/hpc_out/"
folder = "assets/hpc_out/2_6r7m_oj_0_0_os_0_85/"
folder = "assets/hpc_out/3_6r7m_assembled_oj_0_0_os_0_85/"
#folder = "assets/hpc_out/3_6r7m_stacked_oj_0_0_os_0_85/"
folder = "assets/hpc_out/2_6r7m_touching_oj_0_0_os_0_85/"

id_index = 4
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[id_index]) ? parse(Int, split(file, "_")[id_index]) : max
    end
end

evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" in_out_data
        id = parse(Int, split(file, "_")[id_index])
        string = "$(id): $(in_out_data.input.T) | $(in_out_data.input.ε) | $(in_out_data.output.αs[end]) | $(in_out_data.output.Es[1]), $(minimum(in_out_data.output.Es)), $(in_out_data.output.Es[end])"
        evaluation_strings[id] = string
    end
end

In [4]:
for elem in evaluation_strings
    println(elem)
end


2: 1.6 | 0.002 | 0.70292795 | 8185.217724572172, 8146.51163612853, 8163.670588666543

4: 1.6 | 0.002 | 0.74695116 | 8185.217724572172, 8149.261638678713, 8174.181939930744





10: 1.6 | 0.002 | 0.7773067 | 8185.217724572172, 8148.24337637175, 8167.687562980023
11: 1.6 | 0.002 | 0.79029214 | 8185.217724572172, 8154.834930548657, 8164.643799723539
12: 1.6 | 0.002 | 0.6996822 | 8185.217724572172, 8143.90290891926, 8148.174851487759



16: 1.6 | 0.002 | 0.74335593 | 8185.217724572172, 8148.460419434264, 8176.110054149301
17: 1.6 | 0.002 | 0.7592778 | 8185.217724572172, 8153.474031461954, 8173.447083595123
18: 1.6 | 0.002 | 0.8144025 | 8185.217724572172, 8155.913009339745, 8170.651436725451

20: 1.6 | 0.002 | 0.70869136 | 8185.217724572172, 8153.529448080606, 8164.244758019268
21: 1.6 | 0.002 | 0.7859898 | 8185.217724572172, 8151.341839625487, 8173.581023846475

23: 1.6 | 0.002 | 0.57068396 | 8185.217724572172, 8149.22658574359, 8202.201720266045

25: 1.6 | 0.002 | 0.73845714 | 8185.21772

In [5]:
for file in readdir(folder)
    try
        @load "$folder$file" in_out_data
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, in_out_data.input.template_mol), radii, "$(folder)$(id)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
    catch e
        println(e)
        continue
    end
end

In [24]:
@load "assets/hpc_out/3_6r7m_assembled_oj_0_0_os_0_85/3_6r7m_assembled_47_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [33]:
@load "assets/hpc_out/3_6r7m_assembled_oj_0_0_os_0_85/3_6r7m_assembled_47_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))


R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-19.48193, 22.01644, -46.53000]

state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end


permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(MorphoMol.Utilities.sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
    println()
end

1.2873006221221273, 18.550779293943858, 22.747256593704485
14.19511216992349

1.3774676242284738, 18.550779293943858, 22.656875406631716
14.195040774934682

1.2873006221221273, 1.3775764058075557, 2.8189008939530753
1.8279259739609195

18.550888075522934, 1.3775764058075557, 22.656875406631716
14.195113295987403

1.3774676242284738, 1.2874090345163622, 2.8189008939530753
1.8279258508993037

18.550888075522934, 1.2874090345163622, 22.747256593704485
14.195184567914595



(84123.59f0, 20349.809f0, -5968.993f0, 188.49556f0)

In [35]:
@load "assets/hpc_out/3_6r7m_stacked_oj_0_0_os_0_85/3_6r7m_stacked_28_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384  0.803441  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-67.99970, 135.02619, -25.38000]


state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(MorphoMol.Utilities.sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
    println()
end

1.0801358636489278, 1.761615083090717, 2.5839573080281175
1.8085694182559209

3.9681361891428946, 1.761615083090717, 7.226195653808025
4.318648975347212

1.0801358636489278, 1.9392750832172398, 6.0393168036705065
3.019575916845558

5.427780645913811, 1.9392750832172398, 7.226195653808025
4.864417127646359

3.9681361891428946, 5.184605884136937, 6.0393168036705065
5.064019625650112

5.427780645913811, 5.184605884136937, 2.5839573080281175
4.398781279359622



In [38]:
@load "assets/hpc_out/3_6r7m_assembled_oj_0_0_os_0_85/3_6r7m_assembled_47_protor_oj_0_0_os_0_85.jld2" in_out_data
mindex = argmin(in_out_data.output.Es)
in_out_data.output.Vs[mindex], in_out_data.output.As[mindex], in_out_data.output.Ms[mindex], in_out_data.output.Xs[mindex]

(84123.59f0, 20349.809f0, -5968.993f0, 188.49556f0)

In [36]:
@load "assets/hpc_out/3_6r7m_stacked_oj_0_0_os_0_85/3_6r7m_stacked_28_protor_oj_0_0_os_0_85.jld2" in_out_data
mindex = argmin(in_out_data.output.Es)
in_out_data.output.Vs[mindex], in_out_data.output.As[mindex], in_out_data.output.Ms[mindex], in_out_data.output.Xs[mindex]

(85262.81f0, 21122.312f0, -6398.94f0, 62.831852f0)